In [ ]:
import pandas as pd
import numpy as np
from operator import attrgetter

In [ ]:
activity = pd.read_csv('../input/cleoai/cleo_user_activity.csv')
activity.head()

In [ ]:
users = pd.read_csv('../input/cleoai/cleo_users.csv')
users.head()

In [ ]:
#checking data basic info
users.info()
activity.info()

In [ ]:
## data quality and scope check
# 251,817 users created from 2019-01-01 to 2019-08-31, recording activity from 2019-01-01 ro 2020-02-29
#no missing data found
print('user data shape: ', users.shape,
      '\n user count: ', users.id.nunique(),
      '\n data range: ', users.created_at.agg([min,max]))
print('activity data shape: ', activity.shape,
      '\n user count: ', activity.user_id.nunique(),
      '\n data range: ', activity.active_date.agg([min,max]))

# Number of missing values in each column
user_missing_val_count_by_column = (users.isnull().sum())
activity_missing_val_count_by_column = (activity.isnull().sum())
print('user data missing: ', user_missing_val_count_by_column[user_missing_val_count_by_column > 0])
print('activity data missing: ', activity_missing_val_count_by_column[activity_missing_val_count_by_column > 0] )

In [ ]:
#possible duplications in user data
users_dup = users[users.duplicated()]
activity_dup = activity[activity.duplicated()]
user_id_dup = users[users.duplicated('id', keep = False)]

print(users_dup)
print(activity_dup)

In [ ]:
#there are 3 users with duplicated records due to campaign_source. It indicates potential issue with campagin process set up when some one is introduced by a influencer while receiving referral cash
print(user_id_dup)
#at the moment we are lack of business justification of which record should be kept, and the case is very rare. We will take out the duplicated records to be fair.

In [ ]:
#take out the dup records from users
users.drop_duplicates(subset = 'id', keep = False, inplace = True)
activity = activity[~activity.user_id.isin(['930557','1634257','1115108'])]

In [ ]:
#data covering records from Jan 2019 to Aug 2019, 251,820 users were acquired
print('user data shape: ', users.shape,
      '\n user count: ', users.id.nunique(),
      '\n data range: ', users.created_at.agg([min,max]))
print('activity data shape: ', activity.shape,
      '\n user count: ', activity.user_id.nunique(),
      '\n data range: ', activity.active_date.agg([min,max]))

In [ ]:
#get report date for calculations
report_date = pd.to_datetime(activity.active_date.max())

In this code, I will preapre two main tables. 
1. The snapshot table - one row per customer, used for customer profiling, acquisition analysis and major engagment/retention analysis. Enable to understand the user behavior

2. The cohort table - to serve the purpose of cohort retention analysis per market, and also used for MAU analysis.

In [ ]:
## DATA MASSAGING according to the KPI needs
# 1. create the flags for conversion, 1 if converted else 0
users['budget_flag'] = users['budget_created_at'].apply(lambda x: 0 if pd.isna(x) else 1)
users['autosave_flag'] = users['autosave_created_at'].apply(lambda x: 0 if pd.isna(x) else 1)
users['sub_flag'] = users['subscription_created_at'].apply(lambda x: 0 if pd.isna(x) else 1)

In [ ]:
#2. create time lag between created and conversion
users['budget_day_lag'] = (pd.to_datetime(users.budget_created_at)- pd.to_datetime(users.created_at)).dt.days
users['autosave_day_lag'] = (pd.to_datetime(users.autosave_created_at)- pd.to_datetime(users.created_at)).dt.days
users['sub_day_lag'] = (pd.to_datetime(users.subscription_created_at)- pd.to_datetime(users.created_at)).dt.days
users.head()

In [ ]:
# 3.calculating the engagement frequency info from activity table for final snapshot 
# 3.1 calculating the engagement first date, last date, and total activities from activity table 
user_activity_agg = activity.groupby('user_id').agg([min, max, 'count'])
#flattern the result
user_activity_agg.columns = user_activity_agg.columns.get_level_values(1)
user_activity_agg.head()

In [ ]:
#rename the columns and derive retention days
user_activity_agg.rename(columns = {'min':'first_active_date', 'max': 'last_active_date','count' :'total_active_counts' }, inplace = True)
user_activity_agg['retention_days'] = (pd.to_datetime(user_activity_agg.last_active_date)-pd.to_datetime(user_activity_agg.first_active_date)).dt.days
user_activity_agg['retention_months'] = ((pd.to_datetime(user_activity_agg.last_active_date).dt.to_period('M')) \
                                          -(pd.to_datetime(user_activity_agg.first_active_date).dt.to_period('M'))) \
                                         .apply(attrgetter('n'))
user_activity_agg.tail()

In [ ]:
# 3. join to the user table to get the final snapshot table
user_snapshot = user_activity_agg.join(users.set_index('id'), how = 'left') 
user_snapshot.head()

In [ ]:
#checking whether there is any no matching records, all good
user_snapshot.info()

In [ ]:
# tidy up
user_snapshot.rename(columns = {'index': 'id'}, inplace = True)
user_snapshot.head()

In [ ]:
#4. in snapshot table, I also would like to combine the status(Active/dormant/churn) by number of days from the last activity date to reporting date('2020-02')
user_snapshot['days_from_last_active']=(report_date - pd.to_datetime(user_snapshot.last_active_date)).dt.days
user_snapshot['status'] = user_snapshot.days_from_last_active.apply(lambda x: 'Active' if x<31 else ('Dormant' if x < 180 else 'Churn'))
user_snapshot.head()

In [ ]:
## now we are calculating the cohort/MAU. We will use the cohort info in both snapshot table and the cohort table.
#Create monthly cohort analysis data.

#engineering active date monthly level and the cohort info
activity['active_date'] = pd.to_datetime(activity.active_date)
activity['active_month'] = activity.active_date.dt.to_period('M') #transform the date into monthly level
activity['cohort'] = activity.groupby('user_id').active_date.transform('min').dt.to_period('M') #use first active date to identify corhort month
#combine with user data, we will need the country info in it to have market level cohort
user_sub = users[['id','created_at','user_country']].set_index('id')
user_activity_join = user_sub.join(activity.set_index('user_id'), how = 'inner')
user_activity_join.head()

In [ ]:
#tidy up
user_activity_join.reset_index(inplace = True)
user_activity_join.rename(columns = {'index':'user_id'}, inplace = True)
user_activity_join.info

In [ ]:
cohort_chart = user_activity_join[['user_id','cohort', 'active_month','user_country']] \
              .groupby(['cohort', 'active_month','user_country']) \
              .agg(n_customers=('user_id', 'nunique')) \
              .reset_index(drop=False)
cohort_chart['period_number'] = (cohort_chart.active_month - cohort_chart.cohort).apply(attrgetter('n'))
cohort_chart.tail()

In [ ]:
#quickly try out the cohort look (I will rebuild this in the dashboard for easier monitoring)
cohort_pivot = cohort_chart[cohort_chart.user_country == 'US'].pivot_table(index = 'cohort',
                                     columns = 'period_number',
                                     values = 'n_customers')
cohort_size = cohort_pivot.iloc[:,0] #month 0 counts will be the total size of that cohort
retention_cohort = cohort_pivot.divide(cohort_size, axis = 0)
retention_cohort.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
plt.title('Cohorts: User Retention')
sns.heatmap(retention_cohort, mask=retention_cohort.isnull(), annot=True, fmt='.0%');

In [ ]:
retention_cohort.T.plot(figsize=(10,5))
plt.title('Cohorts: User Retention')
plt.xticks(np.arange(1, 12.1, 1))
plt.xlim(0, 12)
plt.ylabel('% of Cohort');

In [ ]:
#looks all good. Output the data
pd.DataFrame.to_csv(user_snapshot,'snapshot.csv', index = False)
pd.DataFrame.to_csv(cohort_chart,'cohort.csv', index = False)